In [1]:
import pandas_helper
import lendingclub_helper
import scikit_helper
import sklearn as sk
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
train_data_file = "finished_train_sample.csv"
df = pandas_helper.readData(train_data_file, lendingclub_helper.dtypes)
df.describe()

In [3]:
# lendingclub_helper.buildDateFeature(df, "hardship_start_date")
# lendingclub_helper.buildFeatures(df)

In [5]:
# feature_columns = lendingclub_helper.getFeatureColumns(df)
# feature_columns

In [5]:
# lendingclub_helper.buildCategorialLabel(df, "loan_status", lendingclub_helper.parse_loan_status)

In [6]:
# label_columns = lendingclub_helper.getLabelColumns(df)
# label_columns

In [7]:
# features = df[feature_columns]
# features.head()

In [8]:
# labels = df[label_columns[0]]
# labels.head()

In [3]:
col = "hardship_start_date"
col = "settlement_date"
lendingclub_helper.buildDateFeature(df, col)
feature_columns = lendingclub_helper.getFeatureColumns(df)
feature_columns
# f_diff = "f_{0}_diff".format(col)
# today = datetime.today()
# df[f_diff] = (today - pd.to_datetime(df[col]))/np.timedelta64(1, "D")
# df[f_diff].fillna(today)
# df[f_diff] = df[f_diff].fillna(df[f_diff].mean())
# set(df[col].fillna("NULL").values)
# pd.to_datetime(df[col]).dt.strftime('%b')


Index(['f_settlement_date_month_Apr', 'f_settlement_date_month_Aug',
       'f_settlement_date_month_Feb', 'f_settlement_date_month_Jan',
       'f_settlement_date_month_Jul', 'f_settlement_date_month_Mar',
       'f_settlement_date_month_May', 'f_settlement_date_month_NULL',
       'f_settlement_date_month_Nov', 'f_settlement_date_month_Oct',
       'f_settlement_date_month_Sep', 'f_settlement_date_month_Jun',
       'f_settlement_date_month_Dec', 'f_settlement_date_diff'],
      dtype='object')

In [6]:
df[feature_columns]

,f_settlement_date_month_Apr,f_settlement_date_month_Aug,f_settlement_date_month_Feb,f_settlement_date_month_Jan,f_settlement_date_month_Jul,f_settlement_date_month_Mar,f_settlement_date_month_May,f_settlement_date_month_NULL,f_settlement_date_month_Nov,f_settlement_date_month_Oct,f_settlement_date_month_Sep,f_settlement_date_month_Jun,f_settlement_date_month_Dec,f_settlement_date_diff
0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,354.843732
1,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.000000
2,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.000000
3,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.000000
4,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.000000
5,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.000000
6,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.000000
7,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.000000
8,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.000000
9,0,0,0,0,0,1,0,0,0,0,0,0.0,0.0,476.843732


In [50]:
v = df[col][0]
pd.to_datetime(v)
# pd.to_datetime(df[col]).dt.strftime('%b')


NaT

In [28]:
df[col].isnull().all()


True

In [22]:
for c in feature_columns:
    try:
        sk.utils.assert_all_finite(df[c].values)
    except Exception as e:
        print(e)
        print(c)
        break

Input contains NaN, infinity or a value too large for dtype('float64').
f_hardship_start_date_diff


In [29]:
# df[["hardship_start_date", "f_hardship_start_date_month_NULL", "f_hardship_start_date_diff"]]
hardship_columns = [c for c in df.columns if "hardship_start_date" in c]
print(hardship_columns)

['hardship_start_date', 'f_hardship_start_date_month_Dec', 'f_hardship_start_date_month_NaT', 'f_hardship_start_date_month_Oct', 'f_hardship_start_date_diff']
